In [3]:
from flair import embeddings as flair_embeddings
from flair.data import Sentence

elmo_model = flair_embeddings.ELMoEmbeddings(model='foo')

In [4]:
sentence = Sentence("I love to code")
elmo_model.embed([sentence])[0].embedding

tensor([])

In [5]:
from allennlp.modules.elmo import Elmo, batch_to_ids

options_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

# Compute two different representation for each token.
# Each representation is a linear weighted combination for the
# 3 layers in ELMo (i.e., charcnn, the outputs of the two BiLSTM))
elmo = Elmo(options_file, weight_file, 2, dropout=0)

# use batch_to_ids to convert sentences to character ids
sentences = [['First', 'sentence', '.'], ['Another', '.']]
character_ids = batch_to_ids(sentences)

embeddings = elmo(character_ids)

In [9]:
from github_search.rnn import print_shapes_recursively

print_shapes_recursively(embeddings['elmo_representations'])

Collection of 2 elements:
	torch.Size([2, 3, 1024])
	torch.Size([2, 3, 1024])
